In [1]:
import requests
import base64

In [2]:
"""
Use authorization credentials to request a temporary access token
"""

CLIENT_ID = '4f570ef647834cd09e37986934a9d6af'
CLIENT_SECRET = '47d53676bc8f4cc6a2814e69e89473ff'

# Base64 encoded credentials
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()


Access token obtained successfully.


In [3]:
%pip install spotipy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [19]:
"""
Get music data from any playlist on Spotify
"""

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Initialize variables for pagination
    offset = 0
    limit = 100
    music_data = []

    while True:

        # Get the tracks from the playlist
        # fields parameter is used to specify the specific track information that is required
        playlist_tracks = sp.playlist_tracks(
            playlist_id, 
            fields='items(track(id, name, artists, album(id, name)))',
            limit=limit,
            offset=offset
        )

        # Check if there are no more tracks to fetch
        if not playlist_tracks['items']:
            break


        for track_info in playlist_tracks['items']:
            track = track_info['track']
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']

            # Get audio features for the track
            audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

            # Get release date of the album
            try:
                album_info = sp.album(album_id) if album_id != 'Not available' else None
                release_date = album_info['release_date'] if album_info else None
            except:
                release_date = None

            # Get popularity of the track
            try:
                track_info = sp.track(track_id) if track_id != 'Not available' else None
                popularity = track_info['popularity'] if track_info else None
            except:
                popularity = None

            # Add additional track information to the track data
            track_data = {
                'Track Name': track_name,
                'Artists': artists,
                'Album Name': album_name,
                'Album ID': album_id,
                'Track ID': track_id,
                'Popularity': popularity,
                'Release Date': release_date,
                'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
                'Explicit': track_info.get('explicit', None),
                'External URLs': track_info.get('external_urls', {}).get('spotify', None),
                'Danceability': audio_features['danceability'] if audio_features else None,
                'Energy': audio_features['energy'] if audio_features else None,
                'Key': audio_features['key'] if audio_features else None,
                'Loudness': audio_features['loudness'] if audio_features else None,
                'Mode': audio_features['mode'] if audio_features else None,
                'Speechiness': audio_features['speechiness'] if audio_features else None,
                'Acousticness': audio_features['acousticness'] if audio_features else None,
                'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
                'Liveness': audio_features['liveness'] if audio_features else None,
                'Valence': audio_features['valence'] if audio_features else None,
                'Tempo': audio_features['tempo'] if audio_features else None,
                # Add more attributes as needed
            }

            music_data.append(track_data)
        
        # Increment the offset by the limit to get the next set of tracks
        offset += limit

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [20]:
# https://open.spotify.com/playlist/1jakGH7uIs1aXNw2UQCgkF?si=69b09a032ac64cef - 20
# https://open.spotify.com/playlist/37i9dQZF1DXdY5tVYFPWb2?si=125e86f3975340a4 - 100
# https://open.spotify.com/playlist/6vI3xbpdPYYJmicjBieLcr?si=c42e61da3acc4d58 - 180
playlist_id = input("Enter the playlist ID: ")

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
music_df.head(5)

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Shape of You,Ed Sheeran,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,7qiZfU4dY1lWllzX7mPBI3,85,2017-03-03,233713,False,https://open.spotify.com/track/7qiZfU4dY1lWllz...,...,0.652,1,-3.183,0,0.0802,0.58100,0.000000,0.0931,0.931,95.977
1,Uptown Funk (feat. Bruno Mars),"Mark Ronson, Bruno Mars",Uptown Special,3vLaOYCNCzngDf8QdBg2V1,32OlwWuMpZ6b0aN2RZOeMS,83,2015-01-12,269667,True,https://open.spotify.com/track/32OlwWuMpZ6b0aN...,...,0.609,0,-7.223,1,0.0824,0.00801,0.000082,0.0344,0.928,114.988
2,Sugar,Maroon 5,Workout 2024,4AEZ921zN1LW6nASbsybdw,6X5Gcn44dNkNz5mGk3gXHB,42,2023-11-08,234827,False,https://open.spotify.com/track/6X5Gcn44dNkNz5m...,...,0.760,1,-7.095,1,0.0349,0.05000,0.000000,0.0859,0.893,120.050
3,Counting Stars,OneRepublic,2014 Best of by uDiscover,6T6DzyYyZ17vNF7hdrpRTm,68gUv1ucWv2Jl5aGP2dOj8,41,2023-08-21,256760,False,https://open.spotify.com/track/68gUv1ucWv2Jl5a...,...,0.741,1,-4.354,0,0.0388,0.07200,0.000000,0.1660,0.454,122.013
4,Ring Ring,MIRA,Ring Ring,6qYGNkUkTjjtK6AeqazTUk,4cSp2hlUlAwgurjIqHN592,58,2024-04-09,150588,False,https://open.spotify.com/track/4cSp2hlUlAwgurj...,...,0.746,2,-3.476,0,0.0541,0.15800,0.000000,0.2570,0.717,102.079


In [21]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [22]:
size = len(music_df)
print(size)

180


In [23]:
import pandas as pd

music_df.to_csv(f'sample_dataset_{size}_songs.csv')